## Search for SLC Sentinel-1 products

In [28]:
import lxml.etree as etree
import requests
import cioppy
ciop = cioppy.Cioppy()

In [29]:
def get_params(osd):

    oss_ns = {'a':'http://www.w3.org/2001/XMLSchema', 
          'b':'http://www.w3.org/2001/XMLSchema-instance',
          'c':'http://a9.com/-/opensearch/extensions/time/1.0/',
          'd':'http://www.opengis.net/eop/2.0',
          'e':'http://purl.org/dc/terms/',
          'f':'http://a9.com/-/spec/opensearch/extensions/parameters/1.0/',
          'g':'http://purl.org/dc/elements/1.1/',
          'h':'http://www.terradue.com/opensearch',
          'i':'http://a9.com/-/opensearch/extensions/geo/1.0/',
          'j':'http://a9.com/-/spec/opensearch/1.1/'}
    
    oss_content = etree.fromstring(requests.get(s1_osd_url).content)
    
    url_template_element = oss_content.xpath('/j:OpenSearchDescription/j:Url[@type="application/atom+xml"]',
                                                 namespaces=oss_ns)[0]
    
    parameters = dict()
    
    for index, parameter in enumerate(url_template_element.xpath('.//f:Parameter', namespaces=oss_ns)):
        
        parameters[parameter.attrib['name']] = {'title' : parameter.attrib['title'], 
                                                'value' : parameter.attrib['value']}
        
        options = []
        for option in parameter.xpath('.//f:Option', namespaces=oss_ns):
            
            options.append(option.attrib['value'])
    
        parameters[parameter.attrib['name']] = {'title' : parameter.attrib['title'], 
                                                'value' : parameter.attrib['value'],
                                                'options' : options}
    return parameters

In [30]:
def get_param_value(osd, os_parameter):

    params = get_params(osd)

    res = None
    
    for index, param in enumerate(params):
        
        if params[param]['value'] == os_parameter:
            
            res = params[param]
            res['name'] = param
     
    return res

Define the Sentinel-1 endpoint

In [31]:
s1_osd_url = 'https://catalog.terradue.com/sentinel1/description'

Get the Sentinel-1 search parameters

In [32]:
s1_parameters = get_params(s1_osd_url)

Print all the OpenSearch parameters

In [38]:
for key, value in s1_parameters.iteritems():

    print '%24s' % s1_parameters[key]['value'], s1_parameters[key]['title'] 

               {geo:uid?} The identifier of the resource within the search engine context (local reference)
    {t2:downloadOrigin?} a string that identifies the download origin (keyword, hostname...) to adapt the enclosure. If the parameter is enclosed between [] (e.g. [terradue]), enclosure will be returned only if there is a enclosure found for this source.
           {startIndex?} index of the first search result desired
            {startPage?} page number of the set of search results desired
       {eop:sensorType?} A string identifying the sensor type
     {eop:accessedFrom?} A string identifying the location from which the resource will be accessed. The catalogue shall return the download location in the enclosure atom link according to the parameter value.
         {t2:landCover?} A number, set or interval requesting the land coverage
      {eop:productType?} A string identifying the product type
            {eop:title?} A name given to the resource
           {time:start?} st

# Find the parameter associated to **'{eop:productType?}'** and its options

In [9]:
product_type_parameter = get_param_value(s1_osd_url, '{eop:productType?}')

In [10]:
product_type_parameter['name']

'pt'

In [19]:
options = s1_parameters[product_type_parameter['name']]['options']

In [20]:
options

['GRD', 'RAW', 'SLC', 'OCN']

Now search for SLC products creating a dictionary with the search parameters

In [21]:
search_params = dict([(product_type_parameter['name'], 
                       options[2])])

In [22]:
search_params

{'pt': 'SLC'}

In [23]:
sensor_type_parameter = get_param_value(s1_osd_url, '{eop:sensorType?}')

In [24]:
sensor_type_parameter['name']

'st'

In [25]:
options = s1_parameters[sensor_type_parameter['name']]['options']

In [26]:
search = ciop.search(end_point=s1_osd_url,
                     params=search_params,
                     output_fields='self,productType,track,enclosure,identifier,wkt,startdate,enddate', 
                     model='EOP')

Show the first result returned

In [15]:
search[0]

{'enclosure': 'https://store.terradue.com/download/sentinel1/files/v1/S1A_IW_SLC__1SDV_20180927T042447_20180927T042504_023879_029B27_4328',
 'enddate': '2018-09-27T04:25:04.8860000Z',
 'identifier': 'S1A_IW_SLC__1SDV_20180927T042447_20180927T042504_023879_029B27_4328',
 'productType': 'SLC',
 'self': 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20180927T042447_20180927T042504_023879_029B27_4328',
 'startdate': '2018-09-27T04:24:47.9700000Z',
 'track': '7',
 'wkt': 'POLYGON((32.490177 62.343678,27.664557 62.775318,28.017311 63.782429,33.011425 63.343391,32.490177 62.343678))'}

You can, of course, set the search parameters directly with:

In [16]:
search = ciop.search(end_point=s1_osd_url,
                     params={'pt': 'SLC'},
                     output_fields='self,productType,track,enclosure,identifier,wkt,startdate', 
                     model='EOP')

In [17]:
search[0]

{'enclosure': 'https://store.terradue.com/download/sentinel1/files/v1/S1A_IW_SLC__1SDV_20180927T042447_20180927T042504_023879_029B27_4328',
 'identifier': 'S1A_IW_SLC__1SDV_20180927T042447_20180927T042504_023879_029B27_4328',
 'productType': 'SLC',
 'self': 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20180927T042447_20180927T042504_023879_029B27_4328',
 'startdate': '2018-09-27T04:24:47.9700000Z',
 'track': '7',
 'wkt': 'POLYGON((32.490177 62.343678,27.664557 62.775318,28.017311 63.782429,33.011425 63.343391,32.490177 62.343678))'}